In [1]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,capacities,lambdas
import numpy as np
from util import welfare_metrics,student_by_col,bayes_update_grade,grades_gr_to_grades_col
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est,anal_cond_exp
from scipy.optimize import fsolve
from scipy.stats import norm, multivariate_normal
from param_def import chi,sigma,lambdas

In [2]:
print(capacities_rate)

[0.2, 0.2]


In [3]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [4]:
def cdf(x, sigma):
    return norm.cdf(x, scale = sigma)

In [5]:
def cdfm(x, y, sigma, rho):
    return multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [6]:
def cdfmsi(x, y, sigma, rho):
    return norm.cdf(y, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [7]:
def cdfmis(x, y, sigma, rho):
    return norm.cdf(x, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [8]:
def market_clear(Pa, Pb, prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    f1 = prop*prefi*(1 - cdf(Pa, sigmai)) + (1 - prop)*prefii*(1 - cdf(Pa, sigmaii)) + prop*(1 - prefi)*cdfmsi(Pa, Pb, sigmai, cori) + (1 -prop)*(1 - prefii)*cdfmsi(Pa, Pb, sigmaii, corii) - capA
    f2 = prop*(1 - prefi)*(1 - cdf(Pb, sigmai)) + (1 - prop)*(1 - prefii)*(1 - cdf(Pb, sigmaii)) + prop*prefi*cdfmis(Pa, Pb, sigmai, cori) + (1 -prop)*prefii*cdfmis(Pa, Pb, sigmaii, corii) - capB
    return f1, f2

In [9]:
def objective_bads(params):
    Pa,Pb = params
    f1,f2 = market_clear(Pa,Pb,prop_gp[0],capacities_rate[0],capacities_rate[1],
                  prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)
    # return np.abs(f1) + np.abs(f2)
    return f1**2+f2**2

In [10]:
# def solv_mc(prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
#     func = lambda P: market_clear(P[0], P[1], prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii)
#     return fsolve(func, x0 = np.array([0, 0]))

# P_A,P_B = solv_mc(prop_gp[0],capacities_rate[0],capacities_rate[1],
#                   prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)


In [11]:
sigma_i = np.sqrt(chi[0]**2 + sigma[0]**2)
sigma_ii = np.sqrt(chi[1]**2 + sigma[1]**2)
cor_i = (chi[0]**2 + lambdas[0]**2)/sigma_i**2
cor_ii = (chi[1]**2 + lambdas[1]**2)/sigma_ii**2

In [12]:
from pybads import BADS

lower_bounds = np.array([-10, -10])
upper_bounds = np.array([10, 10])
plausible_lower_bounds = np.array([-5, -5])
plausible_upper_bounds = np.array([5, 5])
x0 = np.array([0, 0]);        # Starting point
# %%
bads = BADS(objective_bads, x0, lower_bounds, upper_bounds, plausible_lower_bounds, plausible_upper_bounds)
optimize_result = bads.optimize()

# %%
x_min = optimize_result['x']
fval = optimize_result['fval']
print(f"BADS minimum at: x_min = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {optimize_result['func_count']}, time: {round(optimize_result['total_time'], 2)} s")

Beginning optimization of a DETERMINISTIC objective function

 Iteration    f-count         f(x)           MeshScale          Method             Actions
     0           2       0.0345411               1                                 Uncertainty test
     0           6       0.0345411               1         Initial mesh            Initial points
     0          10       0.0345411             0.5         Refine grid             Train
     1          12       0.0253504             0.5     Incremental search (ES-wcm)        
     1          18      0.00138128            0.25         Refine grid             Train
     2          21     0.000644765            0.25     Incremental search (ES-ell)        
     2          26     0.000644765           0.125         Refine grid             
     3          27     0.000150461           0.125     Incremental search (ES-wcm)        
     3          29     5.25757e-07           0.125     Incremental search (ES-ell)        
     3          34     

In [13]:
cutoff_values = list(x_min)

In [14]:
welfare_metrics(cutoff_values,grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.61
Proportion of students in group 0 with only a second preference offer 0.12
Proportion of students in group 0 with first choice offer 0.27
Proportion of students in group 1 with no offer 0.60
Proportion of students in group 1 with only a second preference offer 0.13
Proportion of students in group 1 with first choice offer 0.27


In [15]:
student_by_col(cutoff_values,grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1536
Number of student admitted to college 2 in group 0: 1595
Number of student admitted to college 1 in group 1: 2436
Number of student admitted to college 2 in group 1: 2418


(3972, 4013)

In [16]:
print(capacities)

[4000, 4000]


In [17]:
new_cutoff_values = [1.61886573,0.77066869]
new_cutoff_values

[1.58655643, 0.40073276]

In [20]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,stud_pref,chi,sigma,lambdas,bayes_type='right_partial')

[-3.8704773923138163, 2.555602348307815, 1.7591195462774127, 4.554444290553721, 5.315010545670006, -1.3359115925538283, -2.715562911197226, -0.9233489545357827, -0.06304221054459491, -1.1442727383328894, 4.829581295551452, -2.2599831303851134, 3.6914560686691664, 1.7239765546083383, -0.27522821797721975, -1.0739847363192632, -0.5012500774719478, 3.087272183279321, 1.2838713364117296, -0.9016903756988942, -3.6369167165463328, 3.5600423305798667, 1.2586546477811384, -2.5849673159905393, -0.33627622920398537, -0.13148066469373146, 0.16503668803390514, 0.1797872041071399, -3.670684039680036, 3.2072141623998505, -2.8651293125784263, 2.783687174255623, -1.3409586790732893, 2.7770545922232106, 3.7892862911667535, 3.2523207272817465, 1.6156236899078193, -1.2076146286715348, -0.5992778639029237, 4.3626959570389285, 2.065976926208381, 1.224348708654412, -0.3004859901622776, -2.117511458947486, 3.070939852479645, 0.006359707112277624, 1.8661832975185761, -2.700736903167905, 5.710352770376416, -0.

In [19]:
##    
# grade_estimated should have the following form 
# [[[-3.62656622994722, 2.1811620120023885, 
# -11.563939588668866, 9.522766463940787], 
# [-2.140950614292475, -3.1545339753546977, 
# 0.2686527056082677, -0.6574626041532636, 
# -2.252868772168457, 1.8937660992816987]], 
# [[-5.268780813142571, -0.8191620654894043, 
# -9.60125782299109, 7.9126466724151605], 
# [-3.9756947771758844, -2.6409442009387023, 
# 4.086647894727917, -1.357128391535272, 
# 0.8661128454577303, 0.9586986061854232]]] 
# with 2 groups, 2 colleges, group 1 has 4 students, 
# group 2 has 6 students.
# updated_grade_estimate should have the same form as grade_estimated

In [20]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.61
Proportion of students in group 0 with only a second preference offer 0.12
Proportion of students in group 0 with first choice offer 0.27
Proportion of students in group 1 with no offer 0.60
Proportion of students in group 1 with only a second preference offer 0.13
Proportion of students in group 1 with first choice offer 0.27


In [21]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1535
Number of student admitted to college 2 in group 0: 1581
Number of student admitted to college 1 in group 1: 2432
Number of student admitted to college 2 in group 1: 2415


(3967, 3996)

In [23]:
np.array(stud_pref[0])

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]])

In [57]:
import pandas as pd
import scipy.stats as st

In [58]:
df1 = pd.DataFrame({'A':grade_estimated[0][0],'B':grade_estimated[1][0],'pref':np.array(stud_pref[0]).T[0]})
df2 = pd.DataFrame({'A':grade_estimated[0][1],'B':grade_estimated[1][1],'pref':np.array(stud_pref[1]).T[0]})

In [12]:
df1.loc[df1['pref']==1,'pref_name'] = 'B'
df1.loc[df1['pref']==0,'pref_name'] = 'A'
df1


,A,B,pref,pref_name
0,-3.870477,-2.952409,0,A
1,2.555602,-2.208736,1,B
2,1.759120,-3.065117,1,B
3,4.554444,0.964484,0,A
4,5.315011,0.447377,0,A
...,...,...,...,...
7995,-1.442321,-1.787101,0,A
7996,-2.577367,0.115729,1,B
7997,-3.075432,-6.301088,0,A
7998,0.653565,2.092410,1,B


In [13]:
df2.loc[df2['pref']==1,'pref_name'] = 'B'
df2.loc[df2['pref']==0,'pref_name'] = 'A'
df2

,A,B,pref,pref_name
0,-4.451985,-2.389977,0,A
1,1.655357,0.489349,1,B
2,-0.286189,-2.426861,0,A
3,1.016282,1.160707,0,A
4,-6.176297,-3.145542,1,B
...,...,...,...,...
11995,4.532015,3.858237,0,A
11996,-0.394828,-1.142263,1,B
11997,-1.876270,3.925428,1,B
11998,0.546236,-1.848362,0,A


In [14]:
Pa = 2

In [19]:
df1['bayes'] = np.where(df1['pref_name']=='B',anal_cond_exp(df1['B'],Pa,chi[0],sigma[0],lambdas[0]),df1['B'])
df2['bayes'] = np.where(df2['pref_name']=='B',anal_cond_exp(df2['B'],Pa,chi[1],sigma[1],lambdas[1]),df2['B'])

In [33]:
res1 = st.ecdf(df1['bayes'])
res2 = st.ecdf(df2['bayes'])

In [35]:
from util import grades_col_to_grades_gr

In [36]:
grade_estimated_gr = grades_col_to_grades_gr(grade_estimated)

In [40]:
from util import multivariate_ecdf
Pb = 2

In [41]:
multi_ecdf_gr1 = multivariate_ecdf(grade_estimated_gr[0],[Pa,Pb]).item()
multi_ecdf_gr2 = multivariate_ecdf(grade_estimated_gr[1],[Pa,Pb]).item()
gr1_ecdf_pb = res1.cdf.evaluate(Pb).item()
gr2_ecdf_pb = res2.cdf.evaluate(Pb).item()

In [ ]:
updated_grade_estimate_1 = [anal_cond_exp(i,Pa,chi[0],sigma[0],lambdas[0]) for i in grade_estimated[1][0]]
updated_grade_estimate_2 = [anal_cond_exp(i,Pa,chi[1],sigma[1], lambdas[1]) for i in grade_estimated[1][1]]
res1 = st.ecdf(updated_grade_estimate_1)
res2 = st.ecdf(updated_grade_estimate_2)
updated_grade_estimated[1][0] = updated_grade_estimate_1
updated_grade_estimated[1][1] = updated_grade_estimate_2
grade_estimated_gr = grades_col_to_grades_gr(updated_grade_estimated)
#ECDF
multi_ecdf_gr1 = multivariate_ecdf(grade_estimated_gr[0],[Pa,Pb]).item()
multi_ecdf_gr2 = multivariate_ecdf(grade_estimated_gr[1],[Pa,Pb]).item()
gr1_ecdf_pb = res1.cdf.evaluate(Pb).item()
gr2_ecdf_pb = res2.cdf.evaluate(Pb).item()


In [20]:
[anal_cond_exp(i,Pa,chi[0],sigma[0],lambdas[0]) for i in df2['B']]

[np.float64(-1.3166918797033373),
 np.float64(-0.08292768697735972),
 np.float64(-1.3332201605565457),
 np.float64(0.19170579112687813),
 np.float64(-1.6590527852416657),
 np.float64(2.2542344450780134),
 np.float64(0.45130443488675237),
 np.float64(-1.7474587739598937),
 np.float64(0.45032265997656634),
 np.float64(1.147370562473011),
 np.float64(-1.476990468971769),
 np.float64(0.7439900806987374),
 np.float64(-3.05230383808773),
 np.float64(-0.08966537885866535),
 np.float64(2.3805953404562246),
 np.float64(-1.5320128952508076),
 np.float64(-0.42244322978800064),
 np.float64(-1.2207066592959694),
 np.float64(-0.7112958405322741),
 np.float64(-1.7987927972619664),
 np.float64(-1.7709237242625737),
 np.float64(-0.19526027673864899),
 np.float64(1.232595688622318),
 np.float64(-0.5558417036280395),
 np.float64(1.6615419739326864),
 np.float64(0.4833662669238339),
 np.float64(0.5961147568529039),
 np.float64(0.285256631805971),
 np.float64(0.29225851879943165),
 np.float64(0.42704023911

In [31]:
df2.loc[df2['pref_name'] == 'B','new_val'] = 

,A,B,pref,pref_name
0,-4.451985,-2.389977,0,A
1,1.655357,0.489349,1,B
2,-0.286189,-2.426861,0,A
3,1.016282,1.160707,0,A
4,-6.176297,-3.145542,1,B
...,...,...,...,...
11995,4.532015,3.858237,0,A
11996,-0.394828,-1.142263,1,B
11997,-1.876270,3.925428,1,B
11998,0.546236,-1.848362,0,A
